In [1]:
import folium
import pandas as pd
import numpy as np
import webbrowser
from scipy import spatial
import networkx as nx
import matplotlib.pyplot as plt
from sklearn import decomposition
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
import os
import shutil
from haversine import haversine, Unit

In [2]:
import csv

In [3]:

'''
Simple implementation of the Growing Neural Gas algorithm, based on:
A Growing Neural Gas Network Learns Topologies. B. Fritzke, Advances in Neural
Information Processing Systems 7, 1995.
'''


class GNG:

    def __init__(self, input_data):
        self.network = None
        self.data = input_data
        self.units_created = 0
        plt.style.use('ggplot')

    def find_nearest_units(self, observation):
        distance = []
        for u, attributes in self.network.nodes(data=True):
            vector = attributes['vector']
            # dist = spatial.distance.euclidean(vector, observation)
            dist = haversine(vector,observation)
            distance.append((u, dist))
        distance.sort(key=lambda x: x[1])
        ranking = [u for u, dist in distance]
        return ranking

    def prune_connections(self, a_max):
        for u, v, attributes in self.network.edges(data=True):
            if attributes['age'] > a_max:
                self.network.remove_edge(u, v)
        for u in self.network.nodes():
            if self.network.degree(u) == 0:
                self.network.remove_node(u)

    def fit_network(self, e_b, e_n, a_max, l, a, d, passes=1, plot_evolution=False):
        # logging variables
        accumulated_local_error = []
        global_error = []
        network_order = []
        network_size = []
        total_units = []
        self.units_created = 0
        # 0. start with two units a and b at random position w_a and w_b
        # w_a = [np.random.uniform(-2, 2) for _ in range(np.shape(self.data)[1])]
        # w_b = [np.random.uniform(-2, 2) for _ in range(np.shape(self.data)[1])]
        w_a = self.data[0]
        w_b = self.data[1]
        self.network = nx.Graph()
        self.network.add_node(self.units_created, vector=w_a, error=0)
        self.units_created += 1
        self.network.add_node(self.units_created, vector=w_b, error=0)
        self.units_created += 1

        # 1. iterate through the data
        sequence = 0
        for p in range(passes):
            print('   Pass #%d' % (p + 1))
            steps = 0
            for observation in self.data:
                # 2. find the nearest unit s_1 and the second nearest unit s_2
                nearest_units = self.find_nearest_units(observation)
                s_1 = nearest_units[0]
                s_2 = nearest_units[1]
                # 3. increment the age of all edges emanating from s_1
                for u, v, attributes in self.network.edges_iter(data=True, nbunch=[s_1]):
                    self.network.add_edge(u, v, age=attributes['age']+1)
                # 4. add the squared distance between the observation and the nearest unit in input space
                self.network.node[s_1]['error'] += haversine(observation, self.network.node[s_1]['vector'])**2
                # 5 .move s_1 and its direct topological neighbors towards the observation by the fractions
                #    e_b and e_n, respectively, of the total distance
                update_w_s_1 = e_b * (np.subtract(observation, self.network.node[s_1]['vector']))
                self.network.node[s_1]['vector'] = np.add(self.network.node[s_1]['vector'], update_w_s_1)
                update_w_s_n = e_n * (np.subtract(observation, self.network.node[s_1]['vector']))
                for neighbor in self.network.neighbors(s_1):
                    self.network.node[neighbor]['vector'] = np.add(self.network.node[neighbor]['vector'], update_w_s_n)
                # 6. if s_1 and s_2 are connected by an edge, set the age of this edge to zero
                #    if such an edge doesn't exist, create it
                self.network.add_edge(s_1, s_2, age=0)
                # 7. remove edges with an age larger than a_max
                #    if this results in units having no emanating edges, remove them as well
                self.prune_connections(a_max)
                # 8. if the number of steps so far is an integer multiple of parameter l, insert a new unit
                steps += 1
                if steps % l == 0:
                    if plot_evolution:
                        self.plot_network('./visualization/GNG'+str(sequence) + '.png')
                    sequence += 1
                    # 8.a determine the unit q with the maximum accumulated error
                    q = 0
                    error_max = 0
                    for u in self.network.nodes_iter():
                        if self.network.node[u]['error'] > error_max:
                            error_max = self.network.node[u]['error']
                            q = u
                    # 8.b insert a new unit r halfway between q and its neighbor f with the largest error variable
                    f = -1
                    largest_error = -1
                    for u in self.network.neighbors(q):
                        if self.network.node[u]['error'] > largest_error:
                            largest_error = self.network.node[u]['error']
                            f = u
                    w_r = 0.5 * (np.add(self.network.node[q]['vector'], self.network.node[f]['vector']))
                    r = self.units_created
                    self.units_created += 1
                    # 8.c insert edges connecting the new unit r with q and f
                    #     remove the original edge between q and f
                    self.network.add_node(r, vector=w_r, error=0)
                    self.network.add_edge(r, q, age=0)
                    self.network.add_edge(r, f, age=0)
                    self.network.remove_edge(q, f)
                    # 8.d decrease the error variables of q and f by multiplying them with a
                    #     initialize the error variable of r with the new value of the error variable of q
                    self.network.node[q]['error'] *= a
                    self.network.node[f]['error'] *= a
                    self.network.node[r]['error'] = self.network.node[q]['error']
                # 9. decrease all error variables by multiplying them with a constant d
                error = 0
                for u in self.network.nodes_iter():
                    error += self.network.node[u]['error']
                accumulated_local_error.append(error)
                network_order.append(self.network.order())
                network_size.append(self.network.size())
                total_units.append(self.units_created)
                for u in self.network.nodes_iter():
                    self.network.node[u]['error'] *= d
                    if self.network.degree(nbunch=[u]) == 0:
                        print(u)
            global_error.append(self.compute_global_error())


    def plot_network(self, file_path):
        plt.clf()
        plt.scatter(self.data[:,0], self.data[:,1])
        node_pos = {}
        for u in self.network.nodes_iter():
            vector = self.network.node[u]['vector']
            node_pos[u] = (vector[0], vector[1])
        nx.draw(self.network, pos=node_pos)
        plt.draw()
        plt.savefig(file_path)
        
        
    def get_nodes(self, map1):
        plt.clf()
        node_pos = {}
        nodes=[]
        for u in self.network.nodes_iter():
            vector = self.network.node[u]['vector']
            nodes.append(vector)
            folium.Marker(location=[vector[0], vector[1]], radius = 5, color = 'grey' ).add_to(map1)
        return nodes

    
    def number_of_clusters(self):
        return nx.number_connected_components(self.network)

    
    def cluster_data(self):
        unit_to_cluster = np.zeros(self.units_created)
        cluster = 0
        print('connected nodes are as follows:')
        for c in nx.connected_components(self.network):
            print(c)
            for unit in c:
                unit_to_cluster[unit] = cluster
            cluster += 1
        clustered_data = {}
        for observation in self.data:
            nearest_units = self.find_nearest_units(observation)
            s = nearest_units[0]
            clustered_data.setdefault( unit_to_cluster[s], []).append(observation)
        return clustered_data

    
    def compute_global_error(self):
        global_error = 0
        for observation in self.data:
            nearest_units = self.find_nearest_units(observation)
            s_1 = nearest_units[0]
            global_error += haversine(observation, self.network.node[s_1]['vector'])**2
        return global_error

In [56]:

'''
Simple implementation of the Growing Neural Gas algorithm, based on:
A Growing Neural Gas Network Learns Topologies. B. Fritzke, Advances in Neural
Information Processing Systems 7, 1995.
'''


class GWR:

    def __init__(self, input_data):
        self.network = None
        self.data = input_data
        self.units_created = 0
        plt.style.use('ggplot')

    def find_nearest_units(self, observation):
        distance = []
        for u, attributes in self.network.nodes(data=True):
            vector = attributes['vector']
            # dist = spatial.distance.euclidean(vector, observation)
            dist = haversine(vector,observation)
            distance.append((u, dist))
        distance.sort(key=lambda x: x[1])
        ranking = [u for u, dist in distance]
        return ranking

    def prune_connections(self, a_max):
        for u, v, attributes in self.network.edges(data=True):
            if attributes['age'] > a_max:
                self.network.remove_edge(u, v)
        for u in self.network.nodes():
            if self.network.degree(u) == 0:
                self.network.remove_node(u)

    def fit_network(self, e_b, e_n, a_max, l, a, d, r_min, passes=1, plot_evolution=False):
        # logging variables
        accumulated_local_error = []
        global_error = []
        network_order = []
        network_size = []
        total_units = []
        self.units_created = 0
        # 0. start with two units a and b at random position w_a and w_b
        # w_a = [np.random.uniform(-2, 2) for _ in range(np.shape(self.data)[1])]
        # w_b = [np.random.uniform(-2, 2) for _ in range(np.shape(self.data)[1])]
        w_a = self.data[0]
        w_b = self.data[1]
        self.network = nx.Graph()
        self.network.add_node(self.units_created, vector=w_a, error=0)
        self.units_created += 1
        self.network.add_node(self.units_created, vector=w_b, error=0)
        self.units_created += 1

        # 1. iterate through the data
        sequence = 0
        for p in range(passes):
            print('   Pass #%d' % (p + 1))
            steps = 0
            for observation in self.data:
                # 2. find the nearest unit s_1 and the second nearest unit s_2
                nearest_units = self.find_nearest_units(observation)
                s_1 = nearest_units[0]
                s_2 = nearest_units[1]
                # 3. increment the age of all edges emanating from s_1
                for u, v, attributes in self.network.edges_iter(data=True, nbunch=[s_1]):
                    self.network.add_edge(u, v, age=attributes['age']+1)
                # 4. add the squared distance between the observation and the nearest unit in input space
                dis = haversine(observation, self.network.node[s_1]['vector'])**2
                self.network.node[s_1]['error'] += dis
                # 5 .move s_1 and its direct topological neighbors towards the observation by the fractions
                #    e_b and e_n, respectively, of the total distance
                update_w_s_1 = e_b * (np.subtract(observation, self.network.node[s_1]['vector']))
                self.network.node[s_1]['vector'] = np.add(self.network.node[s_1]['vector'], update_w_s_1)
                update_w_s_n = e_n * (np.subtract(observation, self.network.node[s_1]['vector']))
                for neighbor in self.network.neighbors(s_1):
                    self.network.node[neighbor]['vector'] = np.add(self.network.node[neighbor]['vector'], update_w_s_n)
                # 6. if s_1 and s_2 are connected by an edge, set the age of this edge to zero
                #    if such an edge doesn't exist, and the distance between them is not obviosly hight, create it
                if(haversine(self.network.node[s_1]['vector'],self.network.node[s_2]['vector'])<=r_min):
                    self.network.add_edge(s_1, s_2, age=0)
                # 7. remove edges with an age larger than a_max
                #    if this results in units having no emanating edges, remove them as well
                self.prune_connections(a_max)
                # 8. if distance is greater than a spefic number, insert a new unit
                steps += 1
                if dis >= r_min or  steps % l == 0:
                    if plot_evolution:
                        self.plot_network('./visualization/GNG'+str(sequence) + '.png')
                    sequence += 1
                    # 8. insert a new unit r halfway between the nearest node s_1 and the data point
                    w_r = 0.5 * (np.add(self.network.node[s_1]['vector'], observation))
                    r = self.units_created
                    self.units_created += 1
                    self.network.add_node(r, vector=w_r, error=0)
                    self.network.node[s_1]['error'] *= a
                    self.network.node[r]['error'] = self.network.node[s_1]['error']
                    self.network.add_edge(r, s_1, age=0)
                # 9. decrease all error variables by multiplying them with a constant d
                error = 0
                for u in self.network.nodes_iter():
                    error += self.network.node[u]['error']
                accumulated_local_error.append(error)
                network_order.append(self.network.order())
                network_size.append(self.network.size())
                total_units.append(self.units_created)
                for u in self.network.nodes_iter():
                    self.network.node[u]['error'] *= d
                    if self.network.degree(nbunch=[u]) == 0:
                        print(u)
            global_error.append(self.compute_global_error())


    def plot_network(self, file_path):
        plt.clf()
        plt.scatter(self.data[:,0], self.data[:,1])
        node_pos = {}
        for u in self.network.nodes_iter():
            vector = self.network.node[u]['vector']
            node_pos[u] = (vector[0], vector[1])
        nx.draw(self.network, pos=node_pos)
        plt.draw()
        plt.savefig(file_path)
        
        
    def get_nodes(self, map1):
        plt.clf()
        node_pos = {}
        nodes=[]
        for u in self.network.nodes_iter():
            vector = self.network.node[u]['vector']
            nodes.append(vector)
            folium.Marker(location=[vector[0], vector[1]], radius = 5, color = 'grey' ).add_to(map1)
        return nodes

    
    def number_of_clusters(self):
        return nx.number_connected_components(self.network)

    
    def cluster_data(self):
        unit_to_cluster = np.zeros(self.units_created)
        cluster = 0
        print('connected nodes are as follows:')
        for c in nx.connected_components(self.network):
            print(c)
            for unit in c:
                unit_to_cluster[unit] = cluster
            cluster += 1
        clustered_data = {}
        for observation in self.data:
            nearest_units = self.find_nearest_units(observation)
            s = nearest_units[0]
            clustered_data.setdefault( unit_to_cluster[s], []).append(observation)
        return clustered_data

    
    def compute_global_error(self):
        global_error = 0
        for observation in self.data:
            nearest_units = self.find_nearest_units(observation)
            s_1 = nearest_units[0]
            global_error += haversine(observation, self.network.node[s_1]['vector'])**2
        return global_error

In [78]:
class Test_algorithm:

    def __init__(self, method_type='GNG'):
        # loading data set from a Geolife Trajectories
        data_arr = np.genfromtxt('test_data.plt', delimiter=',', skip_header=6)
        data_df = pd.DataFrame(data=data_arr, columns=['lan', 'long', '*', 'alt', 'time', '**', '***'])
        # first using two dimension langitude and longitude which is easy for visualization and clustering
        self.df = data_df[['lan', 'long']]

    def data_visualization(self, map1=None):
        if (map1 == None):
            map1 = folium.Map(location=self.df.iloc[1], tiles='cartodbpositron', zoom_start=16, )
        self.df.apply(lambda row: folium.CircleMarker(location=[row["lan"], row["long"]], radius=5).add_to(map1),
                      axis=1)
        return map1

    def plot_clusters(self,number_of_clusters, clustered_data, map1):
        colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
                  'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
        for i in range(number_of_clusters):
            if i in clustered_data.keys():
                observations = clustered_data[i]
                observations = np.array(observations)
                for row in observations:
                    folium.CircleMarker(location=[row[0], row[1]], radius=5, color=colors[i]).add_to(map1)
        return map1

    def save_to_csv(self, filename, clustered_data):
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)
            for i in range(len(clustered_data)):
                if i in clustered_data.keys():
                    observations = clustered_data[i]
                    observations = np.array(observations)
                    for row in observations:
                        writer.writerow([str(row[0]), str(row[1]), str(i)])

    def all_points_normal(self, method_type='GNG'):
        # Growing neaural gas algorithm
        if (method_type == 'GNG'):
            data = self.df.values
            gng = GNG(data)
            gng.fit_network(e_b=0.1, e_n=0.01, a_max=1, l=2, a=0.5, d=0.995, passes=1, plot_evolution=False)
            print('Found %d cluster centers.' % gng.number_of_clusters())
            clusters = gng.cluster_data()
            map1 = folium.Map(location=self.df.iloc[1], tiles='cartodbpositron', zoom_start=16, )
            map1 = self.plot_clusters(gng.number_of_clusters(), clusters, map1)
            #             # plot the nodes
            #             nodes = gng.get_nodes(map1)
            #             for vector in nodes:
            #                 folium.Marker(location=[vector[0], vector[1]], radius = 5, color = 'grey' ).add_to(map1)
            display(map1)
            # save the mapping result to csv file
            self.save_to_csv('GNG_result.csv', clusters)
            
        if (method_type == 'GWR'):
            data = self.df.values
            gwr = GWR(data)
            gwr.fit_network(e_b=0.1, e_n=0.01, a_max=1, l=2, a=0.5, d=0.995, r_min=0.25, passes=1, plot_evolution=False)
            print('Found %d cluster centers.' % gwr.number_of_clusters())
            clusters = gwr.cluster_data()
            map1 = folium.Map(location=self.df.iloc[1], tiles='cartodbpositron', zoom_start=16, )
            map1 = self.plot_clusters(gwr.number_of_clusters(), clusters, map1)
            #             # plot the nodes
            #             nodes = gng.get_nodes(map1)
            #             for vector in nodes:
            #                 folium.Marker(location=[vector[0], vector[1]], radius = 5, color = 'grey' ).add_to(map1)
            display(map1)
            # save the mapping result to csv file
            self.save_to_csv('GNG_result.csv', clusters)

    def all_points_withoutlier(self, method_type='GNG'):
        # initialize outliers
        data = self.df
        outlier = {'lan': 1100, 'long': 100}
        data = data.append([outlier] * 10, ignore_index=False).values
        # Growing neaural gas algorithm
        if (method_type == 'GNG'):
            gng = GNG(data)
            gng.fit_network(e_b=0.25, e_n=0.06, a_max=2, l=3, a=0.5, d=1, passes=1, plot_evolution=False)
            print('Found %d cluster centers.' % gng.number_of_clusters())
            clusters = gng.cluster_data()
            map1 = folium.Map(location=data[1], tiles='cartodbpositron', zoom_start=16, )
            map1 = self.plot_clusters(gng.number_of_clusters(), clusters, map1)
            # get nodes and plot
            nodes = gng.get_nodes(map1)
            print(len(nodes))
            for vector in nodes:
                folium.Marker(location=[vector[0], vector[1]], radius=5, color='grey').add_to(map1)
            display(map1)
        # Growing When Required algorithm
        if (method_type == 'GWR'):
            gwr = GWR(data)
            gwr.fit_network(e_b=0.5, e_n=0.1, a_max=3, l=10, a=0.5, d=0.995, r_min=0.3, passes=1, plot_evolution=False)
            print('Found %d cluster centers.' % gwr.number_of_clusters())
            clusters = gwr.cluster_data()
            map2 = folium.Map(location=data[1], tiles='cartodbpositron', zoom_start=16, )
            print(len(clusters))
            map2 = self.plot_clusters(gwr.number_of_clusters(),clusters, map2)
            # plot the nodes 
            nodes = gwr.get_nodes(map2)
            for vector in nodes:
                folium.Marker(location=[vector[0], vector[1]], radius = 5, color = 'grey' ).add_to(map2)
            print(nodes)
            display(map2)
            # save the mapping result to csv file
            self.save_to_csv('GWR_result.csv', clusters)




In [79]:
test = Test_algorithm()
# map1 = test.data_visualization()
# display(map1)

In [8]:
test.all_points_normal('GNG')

   Pass #1
Found 15 cluster centers.
connected nodes are as follows:
{0, 2, 3, 4, 5, 6}
{7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38}
{40, 50, 39}
{41, 52, 47}
{42, 43, 44, 46, 49, 51, 55}
{45, 48, 53, 54, 56}
{64, 65, 130, 99, 132, 142, 57, 58, 59, 60, 61, 62, 63}
{66, 67, 109, 110, 147, 88, 126, 127}
{96, 69, 68, 112}
{128, 131, 133, 134, 135, 139, 143, 144, 70, 71, 72, 73, 89, 90, 91, 92, 94, 98, 100, 103, 104, 119, 120, 121, 123, 124, 125}
{102, 105, 74, 75, 76, 141, 138, 145, 114, 115, 116, 117, 118, 122}
{113, 77, 111}
{108, 78, 79, 146, 95}
{129, 97, 137, 106, 80, 81, 82, 86}
{101, 136, 107, 140, 83, 84, 85, 87, 93}


In [80]:
test.all_points_withoutlier('GWR')

   Pass #1
Found 4 cluster centers.
connected nodes are as follows:
{0, 1, 4, 6, 7, 8, 10, 11, 12, 13, 14, 23, 24, 25, 26, 27, 28, 30, 34, 35, 36, 37, 38, 39, 40}
{3, 2, 5, 29}
{32, 33, 15, 16, 21, 22}
{17, 20, 31}
2
[array([ 84.73207594, 120.46901101]), array([ 31.29683637, 121.54672142]), array([ 31.2962322 , 121.54721954]), array([ 31.29543732, 121.54756025]), array([ 31.29522738, 121.54737445]), array([ 31.29580387, 121.54747486]), array([ 31.29529775, 121.54744467]), array([138.16559193, 119.39247938]), array([138.16479092, 119.39130407]), array([138.165192  , 119.39204298]), array([619.08260557, 109.6961889 ]), array([565.64688606, 110.77285586]), array([ 84.72689389, 120.46983821]), array([138.1623444, 119.3905058]), array([138.16301262, 119.38966548]), array([565.64649443, 110.7713808 ]), array([138.1637506 , 119.38829953]), array([565.64654822, 110.77125462]), array([138.16360093, 119.38871298]), array([565.64635069, 110.7716842 ]), array([565.64590772, 110.77224874]), array([

<Figure size 432x288 with 0 Axes>

In [61]:
test.all_points_normal('GWR')

   Pass #1
Found 13 cluster centers.
connected nodes are as follows:
{1, 2, 145, 146, 147}
{3, 4, 5, 6, 7, 8, 9, 10, 11, 141, 142, 143, 144}
{138, 139, 140, 12, 13, 14, 15, 16, 20, 21, 22}
{137, 17, 19, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 48, 49, 50}
{136, 51, 37, 47}
{52, 38, 135}
{134, 53, 46, 39}
{132, 133, 40, 45, 54}
{128, 129, 130, 131, 41, 42, 43, 44, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 121, 122, 123, 124, 125, 126, 127}
{67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 90, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120}
{96, 97, 84, 85, 86, 88, 95}
{94, 87}
{91, 92, 93}


In [81]:
test.all_points_withoutlier("GNG")

   Pass #1
Found 5 cluster centers.
connected nodes are as follows:
{0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 33, 34, 36, 37, 38}
{1, 50, 51, 52, 53, 54, 55, 57, 59, 66, 68, 70, 75, 77, 87, 89, 90, 95, 96, 99, 100, 101, 102}
{30, 32, 35, 39, 40, 41, 42, 43, 44, 45, 56, 60, 69, 76, 81, 83, 84, 85, 86, 88, 97}
{64, 65, 98, 71, 72, 73, 74, 46, 82, 61, 62, 63}
{67, 78, 79, 80, 47, 49, 48, 58, 91, 92, 93, 94}
103


<Figure size 432x288 with 0 Axes>